In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 70.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b606e7ed21d7290428d169f22d69ba5a0344def15590dfe1c809592c4d432ad1
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install spark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 79 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=b06b53a04996d2e66843c2ad4523dc7871a46eddd03f8f12f0b9da12c9ab21e3
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommandation').getOrCreate()

In [5]:
df = spark.read.csv('/content/drive/MyDrive/Spark with Python/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv', inferSchema=True, header=True)

In [6]:
from pyspark.ml.recommendation import ALS

In [7]:
from pyspark.ml.evaluation import RegressionEvaluator

In [8]:
df.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [9]:
df.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)



In [10]:
df.columns

['movieId', 'rating', 'userId']

In [11]:
df.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [14]:
train, test = df.randomSplit([0.8,0.2])

In [15]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [16]:
model = als.fit(train)

In [17]:
pred = model.transform(test)

In [18]:
pred.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      1|   1.0|    28| -2.5709267|
|      1|   1.0|    26|-0.54931724|
|      3|   1.0|    26|  2.4566326|
|      2|   1.0|    12|   4.326496|
|      0|   1.0|    22|  -2.524905|
|      3|   2.0|    22|  0.5575295|
|      4|   2.0|     1|  2.7624063|
|      0|   1.0|     6| 0.58146536|
|      0|   1.0|     3| -2.0952175|
|      1|   1.0|    19|  1.9904473|
|      2|   3.0|     9|  1.7802036|
|      3|   1.0|     9|  1.9469348|
|      4|   2.0|     8|   1.321235|
|      4|   3.0|    10|   2.692753|
|      1|   3.0|    25|  2.1200306|
|      2|   1.0|    25| 0.88188654|
|      4|   1.0|    29|  0.7915706|
|      0|   1.0|    21|  1.4411675|
|      3|   1.0|    21|    1.92289|
|      3|   1.0|     0| 0.92462313|
+-------+------+------+-----------+
only showing top 20 rows



In [20]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [21]:
rmse = evaluator.evaluate(pred)

In [22]:
print('RMSE')
print(rmse)

RMSE
1.6869660065037255


In [23]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [24]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     12|    11|
|     22|    11|
|     59|    11|
|     61|    11|
|     67|    11|
|     69|    11|
|     72|    11|
|     99|    11|
+-------+------+



In [25]:
recommendations = model.transform(single_user)

In [26]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     12|    11|   2.364836|
|     22|    11|  2.2914522|
|     99|    11|   1.846886|
|     69|    11|  1.7561026|
|     72|    11|  1.5545039|
|     67|    11|   1.075415|
|     59|    11|-0.58923554|
|     61|    11| -1.9305019|
|      9|    11| -2.6086664|
+-------+------+-----------+

